In [1]:
import pandas as pd
import opendatasets as od
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
import matplotlib.pylab as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras import backend as K
!pip3 install keras-tuner --upgrade
!pip3 install keras

Requirement already up-to-date: keras-tuner in /home/apprenant/PycharmProjects/ClimateProject---E3/env/lib/python3.8/site-packages (1.1.2)


In [2]:
od.download("https://www.kaggle.com/saurabhbagchi/ship-and-iceberg-images")

Skipping, found downloaded files in "./ship-and-iceberg-images" (use force=True to force download)


In [3]:
import os
for dirname, _, filenames in os.walk('/home/apprenant/PycharmProjects/ClimateProject---E3/ship-and-iceberg-images/input_data.npz'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

 # Load the data

In [4]:
npz = np.load('/home/apprenant/PycharmProjects/ClimateProject---E3/Data analysis/ship-and-iceberg-images/input_data.npz')

x = npz['X_train']
y = npz['Y_train']
del npz

In [5]:
print('shape of picture: {}'.format(x.shape))
print('shape of picture: {}'.format(y.shape))

shape of picture: (4113, 75, 75, 3)
shape of picture: (4113,)


In [6]:
# First split the data in two sets, 80% for training, 20% for Val/Test)
X_train, X_valtest, y_train, y_valtest = train_test_split(x,y, test_size=0.2, random_state=1, stratify=y)

# Second split the 20% into validation and test sets
X_test, X_val, y_test, y_val = train_test_split(X_valtest, y_valtest, test_size=0.8, random_state=1, stratify=y_valtest)

In [7]:
print(y_test.sum())
print(y_train.sum())

84
1689


In [8]:
print(np.array(X_train).shape)
print(np.array(X_val).shape)
print(np.array(X_test).shape)
print(np.array(y_train).shape)
print(np.array(y_val).shape)
print(np.array(y_test).shape)

(3290, 75, 75, 3)
(659, 75, 75, 3)
(164, 75, 75, 3)
(3290,)
(659,)
(164,)


In [9]:
def build_model(hp):
    # create model object
    model = keras.Sequential([
    #adding first convolutional layer    
    keras.layers.Conv2D(
        #adding filter 
        filters=hp.Int('conv_1_filter', min_value=4, max_value=32, step=4),
        # adding filter size or kernel size
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        #activation function
        activation='relu',
        input_shape=(75,75,3)),
    # adding second convolutional layer 
    keras.layers.Conv2D(
        #adding filter 
        filters=hp.Int('conv_2_filter', min_value=4, max_value=32, step=4),
        #adding filter size or kernel size
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        #activation function
        activation='relu'
    ),
    keras.layers.Dropout(0.4),
    # adding flatten layer    
    keras.layers.Flatten(),
    # adding dense layer    
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=4, max_value=32, step=4),
        activation='relu'
    ),
    keras.layers.Dropout(0.4),
    keras.layers.BatchNormalization(),    
    # output layer    
    keras.layers.Dense(1, activation='softmax')
    ])
    #compilation of model
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='binary_crossentropy',
              metrics=['accuracy'])
    return model

In [10]:
from tensorflow import keras
import keras_tuner

In [11]:
#creating randomsearch object
tuner = keras_tuner.RandomSearch(build_model,
                    objective='val_accuracy',
                    max_trials = 2,
                    overwrite=True)
# search best parameter
tuner.search(X_train,y_train,epochs=4, validation_data=(X_val,y_val))

Trial 1 Complete [00h 02m 03s]
val_accuracy: 0.5144158005714417

Best val_accuracy So Far: 0.5144158005714417
Total elapsed time: 00h 02m 03s

Search: Running Trial #2

Value             |Best Value So Far |Hyperparameter
20                |32                |conv_1_filter
3                 |5                 |conv_1_kernel
4                 |24                |conv_2_filter
3                 |5                 |conv_2_kernel
16                |24                |dense_1_units
0.01              |0.01              |learning_rate

Epoch 1/4
103/103 [==============================] - 9s 82ms/step - loss: 0.6927 - accuracy: 0.5134 - val_loss: 0.6615 - val_accuracy: 0.5144
Epoch 2/4
103/103 [==============================] - 8s 78ms/step - loss: 0.6682 - accuracy: 0.5134 - val_loss: 0.7658 - val_accuracy: 0.5144
Epoch 3/4
103/103 [==============================] - 8s 77ms/step - loss: 0.6347 - accuracy: 0.5134 - val_loss: 0.5818 - val_accuracy: 0.5144
Epoch 4/4
 26/103 [======>.............

KeyboardInterrupt: 

In [ ]:
model=tuner.get_best_models(num_models=1)[0]
#summary of best model
model.summary()

In [ ]:
history = model.fit(X_train,y_train, batch_size=16,
          epochs=5,
          validation_split=0.2,initial_epoch=0)

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

In [ ]:
results = pd.DataFrame(history.history)
results.head()

In [ ]:
import pickle
import tempfile
from tensorflow.keras.models import Sequential, load_model, save_model, Model
from tensorflow.keras.layers import Dense

# Hotfix function
def make_keras_picklable():
    def __getstate__(self):
        model_str = ""
        with tempfile.NamedTemporaryFile(suffix='.hdf5', delete=True) as fd:
            save_model(self, fd.name, overwrite=True)
            model_str = fd.read()
        d = {'model_str': model_str}
        return d

    def __setstate__(self, state):
        with tempfile.NamedTemporaryFile(suffix='.hdf5', delete=True) as fd:
            fd.write(state['model_str'])
            fd.flush()
            model = load_model(fd.name)
        self.__dict__ = model.__dict__


    cls = Model
    cls.__getstate__ = __getstate__
    cls.__setstate__ = __setstate__

# Run the function
make_keras_picklable()

In [ ]:
model = model

In [ ]:
# Saving the model for Future Inferences

model_json = model.to_json()
with open("/home/apprenant/PycharmProjects/ClimateProject---E3/model/model_tuner.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("/home/apprenant/PycharmProjects/ClimateProject---E3/model/model_tuner.h5")